CNN Xray (without pretrained)

In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import numpy as np
from PIL import Image

class XRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                image_name = os.listdir(folder_path)[0]  # Assuming only one image per folder
                if image_name.endswith('.jpg'):
                    self.images.append(os.path.join(folder_path, image_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label
    
 
# Initialize your dataset
xray_dataset = XRayDataset(root_dir='D:/Aspire_xray/xray', transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
]))

In [6]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import random

# Set a seed for reproducibility
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
set_seed(seed)

# CNN classifier model
class CNNClassifier(nn.Module):
    def __init__(self, input_channels=1, num_classes=2):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(64 * 28 * 28, 256)  # Adjust the input features according to your image size
        self.fc2 = nn.Linear(256, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        logits = self.fc2(x)
        return logits

# Function to evaluate the model
def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_probs = []
    all_preds = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to('cuda'), labels.to('cuda')
            logits = model(images)
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)

            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    auc_score = roc_auc_score(all_labels, all_probs)
    accuracy = accuracy_score(all_labels, all_preds)

    return accuracy, auc_score

# Training loop
def train_classifier(model, train_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to('cuda'), labels.to('cuda')

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        train_loss /= len(train_loader.dataset)

# Start of the cross-validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(xray_dataset)), xray_dataset.labels)):

    train_subset = Subset(xray_dataset, train_ids)
    test_subset = Subset(xray_dataset, test_ids)

    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32)

    model = CNNClassifier(input_channels=1, num_classes=2).to('cuda')
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    train_classifier(model, train_loader, criterion, optimizer, epochs=50)

    accuracy, auc_score = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score))

# Calculate mean and STD for each metric across folds
accuracies, aucs = zip(*fold_results)
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)

print(f'Mean Accuracy: {mean_accuracy:.3f}, STD: {std_accuracy:.3f}')
print(f'Mean AUC: {mean_auc:.3f}, STD: {std_auc:.3f}')

Mean Accuracy: 0.675, STD: 0.03
Mean AUC: 0.638, STD: 0.06


ECG only (without pretrained)

In [7]:
import os
import pydicom
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import scipy.signal

def process_dicom(file_path, sampling_rate=500):
    desired_length = 10 * sampling_rate  # 10 seconds of data
    try:
        dicom_data = pydicom.dcmread(file_path)
        if "WaveformSequence" in dicom_data:
            rhythm_waveform = dicom_data.WaveformSequence[1]
            wave_data = rhythm_waveform.get("WaveformData")
            num_channels = rhythm_waveform.NumberOfWaveformChannels
            wave_array = np.frombuffer(wave_data, dtype=np.int16)
            num_samples_per_channel = wave_array.size // num_channels
            
            if wave_array.size % num_channels == 0:
                wave_array = wave_array.reshape(num_samples_per_channel, num_channels)
                

                # Trim or Pad the array to 10 seconds
                if wave_array.shape[0] > desired_length:
                    wave_array = wave_array[:desired_length, :]
                elif wave_array.shape[0] < desired_length:
                    padding = np.zeros((desired_length - wave_array.shape[0], num_channels), dtype=wave_array.dtype)
                    wave_array = np.vstack((wave_array, padding))
                
                # Normalize the array
                wave_array = (wave_array - np.mean(wave_array, axis=0)) / np.std(wave_array, axis=0)

                return wave_array
            else:
                print(f"Unexpected data size in {file_path}. Skipping file.")
                return None
        else:
            print(f"No Waveform data found in {file_path}. Skipping file.")
            return None
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None


class ECGDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.ecg_data = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                for file_name in os.listdir(folder_path):
                    if file_name.endswith('.dcm'):
                        file_path = os.path.join(folder_path, file_name)
                        ecg_waveform = process_dicom(file_path)
                        if ecg_waveform is not None:
                            self.ecg_data.append(ecg_waveform)
                            self.labels.append(label)

    def __len__(self):
        return len(self.ecg_data)

    def __getitem__(self, idx):
        ecg_waveform = self.ecg_data[idx]
        label = self.labels[idx]
        # Reshape waveform to [1, signal_length]
        ecg_waveform = ecg_waveform.reshape(1, -1)  
        return torch.tensor(ecg_waveform, dtype=torch.float32), label

# Usage example
ecg_dataset = ECGDataset(root_dir='D:/Aspire_ecg/ecg')


In [8]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, matthews_corrcoef
import numpy as np
import random
from torch.utils.data import WeightedRandomSampler

def set_seed(seed_value):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

class CNNClassifier(nn.Module):
    def __init__(self, num_classes=2):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)  # Added third convolutional layer
        # Adjust the linear layer input size based on the output size of the last conv layer
        self.fc1 = nn.Linear(64 * 7500, 128)  # Adjusted for the output size before flattening
        self.fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.maxpool(self.relu(self.conv1(x)))
        x = self.maxpool(self.relu(self.conv2(x)))
        x = self.maxpool(self.relu(self.conv3(x)))  # Pass through the third conv layer
        x = x.view(x.size(0), -1)  # Flatten the output
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


# Set a seed value
seed = 42
set_seed(seed)

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = []
mcc_scores = []
auc_scores = []

for fold, (train_idx, test_idx) in enumerate(skf.split(np.zeros(len(ecg_dataset)), ecg_dataset.labels)):

    train_subset = Subset(ecg_dataset, train_idx)
    test_subset = Subset(ecg_dataset, test_idx)

    trainloader = DataLoader(train_subset, batch_size=32, shuffle=True)
    testloader = DataLoader(test_subset, batch_size=32, shuffle=False)

    classifier = CNNClassifier(num_classes=2)  # Adjust 'num_classes' if necessary

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(classifier.parameters(), lr=0.001)

    for epoch in range(50):
        classifier.train()
        for ecg_data, labels in trainloader:

            optimizer.zero_grad()
            outputs = classifier(ecg_data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    classifier.eval()
    true_labels = []
    predictions = []
    probabilities = []

    with torch.no_grad():
        for ecg_data, labels in testloader:

            outputs = classifier(ecg_data)
            predicted = torch.max(outputs.data, 1)[1]
            probas = torch.softmax(outputs, dim=1)[:, 1]

            predictions.extend(predicted.cpu().numpy())
            probabilities.extend(probas.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    fold_accuracy = accuracy_score(true_labels, predictions)
    fold_auc = roc_auc_score(true_labels, probabilities)

    accuracy_scores.append(fold_accuracy)
    auc_scores.append(fold_auc)


average_auc = np.mean(auc_scores)
std_auc = np.std(auc_scores)
average_accuracy = np.mean(accuracy_scores)
std_accuracy = np.std(accuracy_scores)

print(f'Average AUC: {average_auc:.3f}, Std Dev: {std_auc:.3f}')
print(f'Average Accuracy: {average_accuracy:.3f}, Std Dev: {std_accuracy:.3f}')


Mean Accuracy: 0.707, STD: 0.05
Mean AUC: 0.724, STD: 0.05


Xray + ECG (Without pretraining)

In [9]:
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import os
import pydicom
import torch
from torch.utils.data import Dataset


class XRayDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.images = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                image_name = os.listdir(folder_path)[0]  # Assuming only one image per folder
                if image_name.endswith('.jpg'):
                    self.images.append(os.path.join(folder_path, image_name))
                    self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path)
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label
    
 
# Initialize your dataset
xray_dataset = XRayDataset(root_dir='D:/Aspire_xray/xray', transform=transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
    transforms.ToTensor(),
]))


def process_dicom(file_path, sampling_rate=500):
    desired_length = 10 * sampling_rate  # 10 seconds of data
    try:
        dicom_data = pydicom.dcmread(file_path)
        if "WaveformSequence" in dicom_data:
            rhythm_waveform = dicom_data.WaveformSequence[1]
            wave_data = rhythm_waveform.get("WaveformData")
            num_channels = rhythm_waveform.NumberOfWaveformChannels
            wave_array = np.frombuffer(wave_data, dtype=np.int16)
            num_samples_per_channel = wave_array.size // num_channels
            
            if wave_array.size % num_channels == 0:
                wave_array = wave_array.reshape(num_samples_per_channel, num_channels)
                

                # Trim or Pad the array to 10 seconds
                if wave_array.shape[0] > desired_length:
                    wave_array = wave_array[:desired_length, :]
                elif wave_array.shape[0] < desired_length:
                    padding = np.zeros((desired_length - wave_array.shape[0], num_channels), dtype=wave_array.dtype)
                    wave_array = np.vstack((wave_array, padding))
                
                # Normalize the array
                wave_array = (wave_array - np.mean(wave_array, axis=0)) / np.std(wave_array, axis=0)

                return wave_array
            else:
                print(f"Unexpected data size in {file_path}. Skipping file.")
                return None
        else:
            print(f"No Waveform data found in {file_path}. Skipping file.")
            return None
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return None


class ECGDataset(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.ecg_data = []
        self.labels = []

        # Loop through each label directory
        for label in [0, 1]:
            label_dir = os.path.join(root_dir, f'processed_label_{label}')
            for folder_name in os.listdir(label_dir):
                folder_path = os.path.join(label_dir, folder_name)
                for file_name in os.listdir(folder_path):
                    if file_name.endswith('.dcm'):
                        file_path = os.path.join(folder_path, file_name)
                        ecg_waveform = process_dicom(file_path)
                        if ecg_waveform is not None:
                            self.ecg_data.append(ecg_waveform)
                            self.labels.append(label)

    def __len__(self):
        return len(self.ecg_data)

    def __getitem__(self, idx):
        ecg_waveform = self.ecg_data[idx]
        label = self.labels[idx]
        # Reshape waveform to [1, signal_length]
        ecg_waveform = ecg_waveform.reshape(1, -1)  
        return torch.tensor(ecg_waveform, dtype=torch.float32), label

# Usage example
ecg_dataset = ECGDataset(root_dir='D:/Aspire_ecg/ecg')

class CombinedDataset(Dataset):
    def __init__(self, xray_dataset, ecg_dataset):
        self.xray_dataset = xray_dataset
        self.ecg_dataset = ecg_dataset
        assert len(xray_dataset) == len(ecg_dataset), "Datasets must be of the same length."
        
        # Assuming the labels are the same for both datasets and can be directly accessed
        self.labels = [label for _, label in xray_dataset]

    def __len__(self):
        return len(self.xray_dataset)

    def __getitem__(self, idx):
        xray_image, xray_label = self.xray_dataset[idx]
        ecg_waveform, ecg_label = self.ecg_dataset[idx]
        
        # Ensure the labels match if they are supposed to be the same
        assert xray_label == ecg_label, "Labels do not match for the same index."
        
        return xray_image, ecg_waveform, xray_label  # Use either xray_label or ecg_label

    def get_labels(self):
        return self.labels

# Instantiate the combined dataset
combined_dataset = CombinedDataset(xray_dataset, ecg_dataset)
labels = combined_dataset.get_labels()

In [10]:
import torch
import torch.nn as nn


class CombinedCNNClassifier(nn.Module):
    def __init__(self, xray_input_channels=1, ecg_input_channels=1, num_classes=2):
        super(CombinedCNNClassifier, self).__init__()
        # X-ray branch (similar to CNNClassifier for X-ray)
        self.xray_conv1 = nn.Conv2d(xray_input_channels, 16, kernel_size=3, stride=2, padding=1)
        self.xray_conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.xray_conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.xray_flatten = nn.Flatten()
        self.xray_fc = nn.Linear(64 * 28 * 28, 256)  # Adjust according to your X-ray image size

        # ECG branch (similar to CNNClassifier for ECG)
        self.ecg_conv1 = nn.Conv1d(ecg_input_channels, 16, kernel_size=3, stride=1, padding=1)
        self.ecg_conv2 = nn.Conv1d(16, 32, kernel_size=3, stride=1, padding=1)
        self.ecg_conv3 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.ecg_maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.ecg_flatten = nn.Flatten()
        self.ecg_fc = nn.Linear(64 * 7500, 128)  # Adjust according to your ECG signal length

        # Combined layers
        self.combined_fc1 = nn.Linear(256 + 128, 128)  # Combine features from both modalities
        self.combined_fc2 = nn.Linear(128, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, xray, ecg):
        # X-ray branch
        xray = self.relu(self.xray_conv1(xray))
        xray = self.relu(self.xray_conv2(xray))
        xray = self.relu(self.xray_conv3(xray))
        xray = self.xray_flatten(xray)
        xray_features = self.relu(self.xray_fc(xray))

        # ECG branch
        ecg = self.ecg_maxpool(self.relu(self.ecg_conv1(ecg)))
        ecg = self.ecg_maxpool(self.relu(self.ecg_conv2(ecg)))
        ecg = self.ecg_maxpool(self.relu(self.ecg_conv3(ecg)))
        ecg = self.ecg_flatten(ecg)
        ecg_features = self.relu(self.ecg_fc(ecg))

        # Combine features
        combined_features = torch.cat((xray_features, ecg_features), dim=1)
        combined_features = self.dropout(self.relu(self.combined_fc1(combined_features)))
        logits = self.combined_fc2(combined_features)
        return logits

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

def train_model(model, train_loader, criterion, optimizer, epochs=50):
    model.train()  # Set model to training mode
    for epoch in range(epochs):
        running_loss = 0.0
        for xray_images, ecg_signals, labels in train_loader:
            xray_images, ecg_signals, labels = xray_images.to(device), ecg_signals.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = model(xray_images, ecg_signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * xray_images.size(0)
        
        epoch_loss = running_loss / len(train_loader.dataset)


def evaluate_model(model, test_loader):
    model.eval()  # Set model to evaluation mode
    all_labels = []
    all_preds = []
    all_probs = []
    
    with torch.no_grad():
        for xray_images, ecg_signals, labels in test_loader:
            xray_images, ecg_signals, labels = xray_images.to(device), ecg_signals.to(device), labels.to(device)
            
            outputs = model(xray_images, ecg_signals)
            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)
            
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs[:, 1].cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    accuracy = accuracy_score(all_labels, all_preds)
    auc_score = roc_auc_score(all_labels, all_probs)
    return accuracy, auc_score



Using device: cuda


In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score
import numpy as np
import random

# Set seed for reproducibility
def set_seed(seed_value):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed = 42
set_seed(seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(skf.split(np.zeros(len(combined_dataset)), combined_dataset.labels)):
    
    # Splitting the dataset
    train_subset = Subset(combined_dataset, train_ids)
    test_subset = Subset(combined_dataset, test_ids)
    
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_subset, batch_size=32)
    
    # Model initialization
    model = CombinedCNNClassifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Training
    train_model(model, train_loader, criterion, optimizer, epochs=50)
    
    # Evaluation
    accuracy, auc_score = evaluate_model(model, test_loader)
    fold_results.append((accuracy, auc_score))

# Calculate mean and STD for each metric across folds
accuracies, aucs = zip(*fold_results)
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)
mean_auc = np.mean(aucs)
std_auc = np.std(aucs)

print(f'Mean Accuracy: {mean_accuracy:.3f}, STD: {std_accuracy:.3f}')
print(f'Mean AUC: {mean_auc:.3f}, STD: {std_auc:.3f}')



Mean Accuracy: 0.735, STD: 0.039
Mean AUC: 0.748, STD: 0.053
